In [ ]:
import cv2
import numpy as np

In [ ]:
MAX_FEATURES = 200
GOOD_MATCH_PERCENT = 0.15

In [ ]:
def alignImages(im1, im2):
    
    #Convert to grayscale
    im1Gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
    im2Gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)
    
    # Detect ORB features and compute descriptors
    orb = cv2.ORB_create(MAX_FEATURES)
    keypoints1, descriptors1 = orb.detectAndCompute(im1Gray, None)
    keypoints2, descriptors2 = orb.detectAndCompute(im2Gray, None)
    
    #Match features
    matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
    matches = matcher.match(descriptors1, descriptors2, None)
    
    #Sort matches by score
    matches.sort(key = lambda x: x.distance, reverse=True)
    
    # Remove not so good matches
    numGoodMatches = int(len(matches)* GOOD_MATCH_PERCENT)
    matches = matches[:numGoodMatches]
    
    # Draw top matches
    imMatches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
    cv2.imwrite('matches.jpg',imMatches)
    
    # Extract location of good matches
    points1 = np.zeros((len(matches), 2), dtype=np.float32)
    points2 = np.zeros((len(matches), 2), dtype=np.float32)
    
    for i, match in enumerate(matches):
        points1[i, :] = keypoints1[match.queryIdx].pt
        points2[i, :] = keypoints2[match.trainIdx].pt
        
    # Find homography
    h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)
    
    # Use homography
    height, width, channels = im2.shape
    im1Reg = cv2.warpPerspective(im1, h, (width, height))
    
    return im1Reg, h


In [ ]:
refFilename = r"C:\Users\pedro\Python\form.jpg"
imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)

In [ ]:
if __name__ == '__main__':
    
    # Read reference image
    refFilename = r"C:\Users\pedro\Python\form.jpg"
    print("Reading reference image : ", refFilename)
    imReference = cv2.imread(refFilename, cv2.IMREAD_COLOR)
    
    # Read image to be aligned
    imFilename = r"C:\Users\pedro\Python\scanned_form.jpg"
    print("Reading image to align : ", imFilename);
    im = cv2.imread(imFilename, cv2.IMREAD_COLOR)
    
    print("Aligning images ...")
    # Registered image will be resotred in imReg.
    # The estimated homography will be stored in h.
    imReg, h = alignImages(im, imReference)
    
    # Write aligned image to disk.
    outFilename = "aligned.jpg"
    print("Saving aligned image : ", outFilename);
    cv2.imwrite(outFilename, imReg)
    
    # Print estimated homography
    print("Estimated homography : \n",  h)

In [ ]:
import cv2       # OpenCV

# 1. Load the original image
img = cv2.imread('form.jpg')
# Convert the image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 2. Create BRISK algorithm
# OpenCV default threshold = 30, octaves = 3
# Using 4 octaves as cited as typical value by the original paper by Leutenegger et al.
# Using 70 as detection threshold similar to real-world example of this paper
brisk = cv2.BRISK_create(70, 4)

# 3. Combined call to let the BRISK implementation detect keypoints
# as well as calculate the descriptors, based on the grayscale image.
# These are returned in two arrays.
(kps, descs) = brisk.detectAndCompute(gray, None)

# 4. Print the number of keypoints and descriptors found
print("# keypoints: {}, descriptors: {}".format(len(kps), descs.shape))
# To verify: how many bits are contained in a feature descriptor?
# Should be 64 * 8 = 512 bits according to the algorithm paper.
print(len(descs[1]) * 8)

# 5. Use the generic drawKeypoints method from OpenCV to draw the 
# calculated keypoints into the original image.
# The flag for rich keypoints also draws circles to indicate
# direction and scale of the keypoints.
imgBrisk = cv2.drawKeypoints(gray, kps, img, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# 6. Finally, write the resulting image to the disk
cv2.imwrite('brisk_keypoints.jpg', imgBrisk)

In [13]:
import cv2
import numpy as np


def get_corrected_img(img1, img2,MIN_MATCHES = 50):
    MIN_MATCHES = 50

    orb = cv2.ORB_create(nfeatures=500)
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    index_params = dict(algorithm=6,
                        table_number=6,
                        key_size=12,
                        multi_probe_level=2)
    search_params = {}
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # As per Lowe's ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) > MIN_MATCHES:
        src_points = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_points = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        m, mask = cv2.findHomography(src_points, dst_points, cv2.RANSAC, 5.0)
        corrected_img = cv2.warpPerspective(img1, m, (img2.shape[1], img2.shape[0]))
        print(True)
        return corrected_img
    print(False)
    return img2


if __name__ == "__main__":
    im1 = cv2.imread('form2.jpg')
    im2 = cv2.imread('scanned_form2.jpg')

    img = get_corrected_img(im1,im2,100)
    cv2.imwrite('corimg.jpg', img)

False


In [52]:
import cv2
import imutils

temp1 = cv2.imread('form2.jpg', cv2.IMREAD_GRAYSCALE)
temp2 = cv2.imread('scanned_form2.jpg', cv2.IMREAD_GRAYSCALE)

temp11 = cv2.medianBlur(temp1,13)
temp22 = cv2.medianBlur(temp2,13)

img1 = cv2.Canny(temp11,100,70)
img2 = cv2.Canny(temp22,100,70)

orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(temp11, None)
kp2, des2 = orb.detectAndCompute(temp22, None)

# matcher takes normType, which is set to cv2.NORM_L2 for SIFT and SURF, cv2.NORM_HAMMING for ORB, FAST and BRIEF
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1, des2)
matches = sorted(matches, key=lambda x: x.distance)# draw first 50 matches
match_img = cv2.drawMatches(temp11, kp1, temp22, kp2, matches[:100], None)

# Downsize and maintain aspect ratio
showimg = imutils.resize(match_img, height=600)

cv2.imshow('corrected_form.jpg',showimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
can = cv2.Canny(img2,200,100)

cv2.imshow('can',can)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
tmp1 = cv.imread('mask.jpg',cv.IMREAD_GRAYSCALE)          # queryImage
tmp2 = cv.imread('scanned_form.jpg',cv.IMREAD_GRAYSCALE) # trainImage

ret,img1 = cv2.threshold(tmp1,100,255,cv2.THRESH_BINARY)
ret,img2 = cv2.threshold(tmp2,100,255,cv2.THRESH_BINARY)

# Initiate SIFT detector
sift = cv.SIFT_create()
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(des1,des2,k=2)
# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.6*n.distance:
        matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv.DrawMatchesFlags_DEFAULT)
img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,matches,None,**draw_params)

# Downsize and maintain aspect ratio
showimg = imutils.resize(img3, height=600)

cv2.imshow('corrected_form.jpg',showimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [68]:
img1 = cv.imread('scanned_form.jpg')          # queryImage# convert the image to grayscale and blur it slightly
gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (11, 11), 0)
ret,thresh1 = cv2.threshold(gray,100,255,cv2.THRESH_BINARY)
cv2.imshow('can',thresh1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [77]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml') 


faces = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
eyes = cv2.CascadeClassifier('haarcascade_eye.xml') 

In [74]:
def process_img(image):
    original_image = image
    processed_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    processed_img =  cv2.Canny(processed_img, threshold1 = 200, threshold2=300)
    return processed_img

In [89]:
frame = cv.imread('scanned_form.jpg')
gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
face=faces.detectMultiScale(frame,1.3,5)

# now getting into the face and its position
for (x,y,w,h) in face:
    # drawing the rectangle on the face
    cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),thickness=4)

    # now the eyes are on the face
    # so we have to make the face frame gray
    gray_face=gray_frame[y:y+h,x:x+w]

    # make the color face also
    color_face=frame[y:y+h,x:x+w]

    # check the eyes on this face
    eye=eyes.detectMultiScale(gray_face,1.3,5)
    print(eye)
    # get into the eyes with its position
    for (a,b,c,d) in eye:
        # we have to draw the rectangle on the
        # coloured face
        print(a+((c-a)/2),b+((d-b)/2))
        cv2.rectangle(color_face,(a,b),(a+c,b+d),(0,255,0),thickness=4)

    # show the frame
# Downsize and maintain aspect ratio
showimg = imutils.resize(frame, height=500)

cv2.imshow('corrected_form.jpg',showimg)
cv2.waitKey(0)
cv2.destroyAllWindows()

[[ 37  44  44  44]
 [112  68  41  41]]
40.5 44.0
76.5 54.5


In [ ]:
a+((c-a)/2),b+((d-b)/2)